![Attention](pic/attention.PNG)

### Freezing attention head <br>
Freeze only QKV matrices?
OR
Also projection layer ($W_{o}$)

In [8]:
from transformers.modeling_gpt2 import *

In [10]:
from transformers.configuration_gpt2 import *

In [15]:
import torch.nn as nn

In [195]:
def printgradnorm(self, grad_input, grad_output):
    print('Inside ' + self.__class__.__name__ + ' backward')
    print('Inside class:' + self.__class__.__name__)
    print('')
    print('grad_input: ', type(grad_input))
    print('grad_input[0]: ', type(grad_input[0]))
    print('grad_output: ', type(grad_output))
    print('grad_output[0]: ', type(grad_output[0]))
    print('')
    print('grad_input size:', grad_input[0].size())
    print('grad_input :', grad_input[0])
    print('grad_output size:', grad_output[0].size())
    print('grad_output :', grad_output[0])
    print('grad_input norm:', grad_input[0].norm())
attn_model.attention.c_attn.register_backward_hook(printgradnorm)

In [241]:
def freeze_attention_head(layer, head_indices, bias_indices=None):
    if bias_indices is None:
        bias_indices = head_indices

    def freezing_hook_full(layer, grad_input, grad_output, weight_multiplier, bias_multiplier):
        return (grad_input[0] * weight_multiplier,)

    weight_multiplier = torch.ones(layer.weight.shape[1])
    for head in head_indices:
        weight_multiplier[head*64:(head+1)*64] = 0
        weight_multiplier[head*64+768:(head+1)*64+768] = 0
        weight_multiplier[head*64+(768*2):(head+1)*64+(768*2)] = 0
#     weight_multiplier = weight_multiplier.view(1, -1)
    bias_multiplier = torch.ones(layer.weight.shape[1])
    bias_multiplier[bias_indices] = 0
    freezing_hook = lambda layer, grad_input, grad_output: freezing_hook_full(layer, grad_input, grad_output, weight_multiplier, bias_multiplier)

    layer.register_backward_hook(freezing_hook)

In [242]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.cfg = GPT2Config()
        self.attention = Attention(nx=768,n_ctx=1024,config=self.cfg)
        self.lmhead = nn.Linear(self.cfg.n_embd, self.cfg.vocab_size, bias=False)
    def forward(self, x):
        x = self.attention(x)
        return self.lmhead(x[0])

In [243]:
attn_model = Model()

In [244]:
loss_fct = nn.NLLLoss()
optimizer = torch.optim.Adam(attn_model.parameters(), lr=0.001)

In [245]:
freeze_attention_head(attn_model.attention.c_attn,[0])

In [246]:
random_input = torch.rand((1,16,768),dtype=torch.float)
random_target = torch.rand((16,50257),dtype=torch.float)
random_target = random_target.argmax(dim=1)

In [249]:
model_output = attn_model(random_input)
print(attn_model.attention.c_attn.weight.shape)
print(attn_model.attention.c_attn.weight[:,0].shape)
old_attn_wgt = attn_model.attention.c_attn.weight.clone().detach()
optimizer.zero_grad()
loss = loss_fct(model_output[0],random_target)
attn_model.zero_grad()
loss.backward()
optimizer.step()
new_attn_wgt = attn_model.attention.c_attn.weight.clone().detach()

torch.Size([768, 2304])
torch.Size([768])


In [255]:
old_attn_wgt[:,:64].equal(new_attn_wgt[:,:64])

True

In [256]:
old_attn_wgt[:,64:768].equal(new_attn_wgt[:,64:768])

False

In [257]:
old_attn_wgt[:,768:64+768].equal(new_attn_wgt[:,768:64+768])

True

In [258]:
old_attn_wgt[:,768+64:128+768].equal(new_attn_wgt[:,768+64:128+768])

False

In [164]:
inp = torch.ones(2,3)
inp

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [165]:
inp * torch.tensor([1,0,1])

tensor([[1., 0., 1.],
        [1., 0., 1.]])